## ✅ Script Transfer Data PostgreSQL to HDFS

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import os
import psycopg2 as pg
from datetime import datetime

In [ ]:
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['JAVA_HOME'] = '/usr/local/jdk8u222-b10'
os.environ['HADOOP_USER_NAME']='hive'
os.environ['PYSPARK_PYTHON'] ='/HDFS01/anaconda3/envs/main/bin/python'

conf = pyspark.SparkConf().setAll([
     ('spark.driver.maxResultSize', '0'),
     ('spark.driver.memory', '4g'),
     ('spark.sql.repl.eagerEval.enabled','true'),
     ('hive.strict.managed.tables','false'),
     ('hive.metastore.uris', 'thrift://nn01.bigdata:9083'),
     ('metastore.client.capability.check','false')
    ])
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("myApp") \
        .config(conf=conf) \
        .enableHiveSupport() \
        .getOrCreate();

In [ ]:
def getDate():
    now = datetime.now()
    today = now.strftime("%Y%m%d_%H%M%S")
    return today

#### 🟢 Postgres Connector ```pip install psycopg2-binary```

In [ ]:
def pgCon():
    pgconn = pg.connect(database="crawler",
                            host="192.168.10.22",
                            user="softnix",
                            password="softnixteam",
                            port="5433")
    return pgconn

#### 🟢 ดึงข้อมูลตารางทั้งหมด

In [ ]:
sql = """
    SELECT schemaname, tablename
    FROM pg_catalog.pg_tables
    WHERE schemaname != 'pg_catalog' AND 
        schemaname != 'information_schema';
"""
df = pd.read_sql(sql, con=pgCon())
df

#### 🟢 Loop ทำการ insert ทุกๆ Table โดย PySpark

In [ ]:
for i,v in df.iterrows():
    try:
        df2 = pd.read_sql(f"select * from {v['tablename']}", con=pgCon())
        df2.to_csv("/tmp/export.csv", index=False, header=True)
        output=spark.read.csv("file:///tmp/export.csv", inferSchema=True, header=True)
        output.write \
          .mode("overwrite") \
          .saveAsTable(f"pyspark.{v['tablename']}")
        os.system("rm -f /tmp/export.csv")
        print("Success:", v['tablename'])
    except Exception as e:
        print("Skiped because to Data in files")
        continue

#### 🟢 Use OS validate to list files in HDFS

In [ ]:
os.system("sudo -u hdfs hdfs dfs -ls /user/hive/warehouse/pyspark.db/crawler_site")